In [43]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta



In [44]:
# Số lượng bản ghi
num_records = 5000

# Tạo customer_id
def generate_unique_customer_ids(num_ids):
    all_customer_ids = np.arange(10000000, 100000000)
    np.random.shuffle(all_customer_ids)
    return all_customer_ids[:num_ids]

# Sinh ra 50 customer_id không trùng lặp
customer_ids = generate_unique_customer_ids(num_records)



In [45]:
# Tạo full_name ngẫu nhiên
def generate_random_name():
    first_names = ['John', 'Jane', 'Robert', 'Emily', 'Michael', 'Emma', 'William', 'Olivia', 'James', 'Ava',
               'Sophia', 'Alexander', 'Isabella', 'Daniel', 'Mia', 'David', 'Charlotte', 'Joseph', 'Abigail',
               'Andrew', 'Harper', 'Joshua', 'Evelyn', 'Matthew', 'Amelia', 'Ryan', 'Elizabeth', 'Ethan', 'Sofia']

    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller', 'Davis', 'Martinez', 'Lee',
              'Rodriguez', 'Wilson', 'Martinez', 'Anderson', 'Taylor', 'Thomas', 'Hernandez', 'Moore', 'Martin',
              'Jackson', 'Thompson', 'White', 'Lopez', 'Gonzalez', 'Harris', 'Clark', 'Lewis', 'Robinson', 'Walker']

    return f"{random.choice(first_names)} {random.choice(last_names)}"

full_names = [generate_random_name() for _ in range(num_records)]



In [46]:
# Tạo target_code theo tỷ lệ 1-2-7
target_codes = np.random.choice([1000, 2000, 4000], num_records, p=[0.1, 0.2, 0.7])



In [47]:
# Tạo gender theo tỷ lệ 4-6
genders = np.random.choice(['MALE', 'FEMALE'], num_records, p=[0.4, 0.6])




In [48]:
# Tạo ngày sinh (dob) ngẫu nhiên từ 1/1/1970 đến 31/12/2005
start_date = datetime(1970, 1, 1)
end_date = datetime(2005, 12, 31)

dob_dates = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(num_records)]
dob_strings = [date.strftime('%Y-%m-%d') for date in dob_dates]



In [49]:
# Tạo segment theo tỷ lệ 2-3-5
segments = np.random.choice(['KHTT', 'KHCL', 'KHUT'], num_records, p=[0.2, 0.3, 0.5])



In [50]:
# Tạo DataFrame từ các dữ liệu đã tạo
data = {
    'customer_id': customer_ids.astype(str),
    'full_name': full_names,
    'target_code': target_codes,
    'gender': genders,
    'dob': dob_strings,
    'segment': segments
}



In [51]:
df = pd.DataFrame(data)

# In thông tin mẫu của DataFrame
print(df.head())

  customer_id     full_name  target_code  gender         dob segment
0    63205802  Daniel Lewis         4000  FEMALE  1981-11-30    KHUT
1    83990555   James Lewis         4000    MALE  1996-04-04    KHUT
2    62980626    John Davis         4000  FEMALE  1995-07-02    KHCL
3    59803307  Emily Garcia         4000    MALE  1988-12-07    KHUT
4    67706131   James White         4000  FEMALE  1985-07-27    KHUT


In [52]:
# Số lượng bản ghi trong account
num_records_account = 8000

# Tạo account_number là chuỗi ký tự số có độ dài 12
account_numbers = [''.join(random.choices('0123456789', k=12)) for _ in range(num_records_account)]


In [53]:
# Sử dụng tập customer_id đã tạo ở trước đó để tạo customer_id cho account
customer_ids_account = np.random.choice(df['customer_id'], num_records_account)


In [54]:
# Tạo opening_date ngẫu nhiên từ 1/1/2020 đến 31/5/2024
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 5, 31)

opening_dates = [start_date + timedelta(days=random.randint(0, (end_date - start_date).days)) for _ in range(num_records_account)]
opening_date_strings = [date.strftime('%Y-%m-%d') for date in opening_dates]


In [55]:
# Tạo product_id gồm 1001 và 1023 theo tỷ lệ 7-3
product_ids = np.random.choice([1001, 1023], num_records_account, p=[0.7, 0.3])


In [56]:
# Tạo currency gồm VND, USD, EUR theo tỷ lệ 8-1.5-0.5
currencies = np.random.choice(['VND', 'USD', 'EUR'], num_records_account, p=[0.8, 0.15, 0.05])


In [57]:
# Tạo DataFrame account từ các dữ liệu đã tạo
data_account = {
    'account_number': account_numbers,
    'customer_id': customer_ids_account,
    'opening_date': opening_date_strings,
    'product_id': product_ids,
    'currency': currencies
}


In [58]:
df_account = pd.DataFrame(data_account)

# In thông tin mẫu của DataFrame account
print(df_account.head())

  account_number customer_id opening_date  product_id currency
0   744761900043    58000981   2022-02-14        1001      VND
1   555286461200    60035164   2023-03-23        1001      VND
2   539668218911    89506712   2023-06-29        1001      VND
3   482534465189    25136560   2023-05-24        1001      VND
4   119790767773    23819437   2022-04-03        1001      VND


In [59]:
import psycopg2
from psycopg2 import sql

# Thông tin kết nối PostgreSQL
dbname = "postgres"
user = "postgres"
password = "login1234"
host = "localhost"
port = "5432"

In [71]:
# Kết nối đến PostgreSQL
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Cursor để thực thi các lệnh SQL
cursor = conn.cursor()

# Tên bảng trong PostgreSQL
table_name = "dwh.customer"

# cursor.execute("select * from dwh.target")
# rows=cursor.fetchall()
# print(rows)
# Lặp qua từng dòng trong DataFrame và chèn vào bảng
for index, row in df.iterrows():
    insert_query = sql.SQL("INSERT INTO dwh.customer ({}) VALUES ({})").format(
        # sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, df.columns)),
        sql.SQL(', ').join(map(sql.Literal, row))
    )
    cursor.execute(insert_query)

# Lưu thay đổi và đóng kết nối
conn.commit()
cursor.close()
conn.close()

UndefinedColumn: column "account_number" of relation "customer" does not exist
LINE 1: INSERT INTO dwh.customer ("account_number", "customer_id", "...
                                  ^


In [72]:
# Kết nối đến PostgreSQL
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Cursor để thực thi các lệnh SQL
cursor = conn.cursor()

# Tên bảng trong PostgreSQL
table_name = "dwh.account"

# cursor.execute("select * from dwh.target")
# rows=cursor.fetchall()
# print(rows)
# Lặp qua từng dòng trong DataFrame và chèn vào bảng
for index, row in df_account.iterrows():
    insert_query = sql.SQL("INSERT INTO dwh.account ({}) VALUES ({})").format(
        # sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, df_account.columns)),
        sql.SQL(', ').join(map(sql.Literal, row))
    )
    cursor.execute(insert_query)

# Lưu thay đổi và đóng kết nối
conn.commit()
cursor.close()
conn.close()

In [62]:
# spark = SparkSession.builder \
#     .appName("Import Customer DataFrame to PostgreSQL") \
#     .config("spark.jars", "postgresql-42.6.2.jar") \
#     .getOrCreate()

In [63]:
# customer_df = spark.createDataFrame(df)


In [64]:
# customer_df.printSchema

In [65]:
# data_schema = [
#     StructField('customer_id', IntegerType(),True), 
#     StructField('full_name', StringType(), True),
#     StructField('target_code', DateType(), True),
#     StructField('gender', DoubleType(), True),
#     StructField('dob', TimestampType(), True),
#     StructField('segment', DoubleType(), True)
# ]

In [66]:
# from pyspark.sql.functions import col

# # Áp dụng schema mới vào customer_df
# customer_df = customer_df.withColumn("customer_id", col("customer_id").cast(StringType())) \
#     .withColumn("full_name", col("full_name").cast(StringType())) \
#     .withColumn("target_code", col("target_code").cast(IntegerType())) \
#     .withColumn("gender", col("gender").cast(StringType())) \
#     .withColumn("dob", col("dob").cast(TimestampType())) \
#     .withColumn("segment", col("segment").cast(StringType()))

In [67]:
# customer_df.printSchema()

In [68]:
# customer_df.select('customer_id').show(5)

In [69]:
# customer_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/postgres',
#      dbtable='dwh.customer',
#      user='postgres',
#      password='login1234',
#      driver='org.postgresql.Driver',
#      schema='dwh').\
#      mode("append").\
#      save()

# # jdbc_url = "jdbc:postgresql://localhost:5432/postgres"
# # db_properties = {
# #     "user": "postgres",
# #     "password": "login1234",
# #     "driver": "org.postgresql.Driver",
# #     "connectTimeout": "30000",  # Timeout kết nối là 30 giây
# #     "socketTimeout": "30000"
# # }
# # customer_df.write.jdbc(url=jdbc_url, table="customer", mode="append", properties=db_properties)

In [70]:
spark.stop()